In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from gensim_modeling import *
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict
from pandas_ml import ConfusionMatrix
from sklearn.metrics import precision_score, recall_score, accuracy_score
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Autotagging with Doc2Vec

Doc2Vec can learn tag sequences as they cooccur with with document text. Therefore, all of the tags in our dataset have an embeding in the Doc2Vec word embedding space.

A naive approach to quantizing a document is to take the average of the word vectors in that document and assign the average to the document.

In regards to autotagging, the idea is to quantize a document, search for k-nearest neighbors in the word embedding space, and filter for labeled tags with a cosine similiarity criteria.

<br>
![](images/2_pass_IR_system.png)

In [2]:
balanced_classes_file = open("pickle_jar/dataframes/classify_Stopwords_3xTitle_ner_stripPunct_dataframe.p", 'rb')
balanced_classes_data = pickle.load(balanced_classes_file)

balanced_classes_tag_counter_file = open('pickle_jar/tag_counters/classify_Stopwords_3xTitle_ner_stripPunct_tag_counts.p', 'rb')
balanced_classes_tag_counter = pickle.load(balanced_classes_tag_counter_file)

balanced_classes_bow_counter_file = open('pickle_jar/post_counters/classify_Stopwords_3xTitle_ner_stripPunct_token_counts.p', 'rb')
balanced_classes_bow_counter = pickle.load(balanced_classes_bow_counter_file)


thread_dict = {'biology':0,'cooking':1,'crypto':2,'diy':3,'robotics':4,'travel':5}
balanced_classes_data['thread_ids'] = balanced_classes_data.thread.apply(lambda x: thread_dict[x]).values

In [3]:
d2v_model = doc2vec.Doc2Vec.load('doc2vec_models/balanced.doc2vec')


In [4]:
tag_predictions = auto_tag(d2v_model, 300, balanced_classes_data, 500, balanced_classes_tag_counter)

In [6]:
true_tags = balanced_classes_data['tags'].values

In [34]:
def view_example_post(index):
    print("QUESTION:\n{}\n".format(balanced_classes_data['title'].iloc[index]))

    print("ELABORATION:\n{}\n".format(balanced_classes_data['content'].iloc[index]))

    print("TAGS:\n{}\n".format(balanced_classes_data['tags'].iloc[index]))
    
    print("AUTOTAGS:\n{}\n".format(tag_predictions[index]))

    print("CLASS: ***{}***".format(balanced_classes_data['thread'].iloc[index]))


view_example_post(np.random.randint(0,len(balanced_classes_data)))

QUESTION:
Additional Power to DC Motor via Second Power Source

ELABORATION:
<p>How can I provide more power to a DC motor that is in series behind a receiver circuit hacked out of a cheap RC car without burning up the receiver board? The board runs off two AAs at about 3V. I'm replacing the stock motor with a slightly larger one (12V, taken from a printer) and remounting it on a chassis for a homebrew robotics project... just messing around to learn more. I imagine I could go safely to 4.5V or even 6V with the receiver but I don't want to go much higher since half the stuff is epoxied and I can't really tell what's in there.</p>

<p>What I'd like to be able to do is add an additional two AA batteries behind the receiver to run the receiver system at 6V but add another two 3V 123A batteries to have the motor at 12V with the ability to run with the higher current draw due to the heavier load the motor will handle on its fancy new chassis... but without pulling that current through the r

In [25]:
def evaluate_autotag(ytrue,ypred):
    tprs = []
    fprs = []
    for t,p in zip(ytrue, ypred):
        row_tps = 0
        row_fps = 0
        for tkn in t:
            if tkn in p:
                row_tps += 1
            else:
                row_fps += 1
#         tpr = row_tps/len(ytrue)
#         fpr = row_fps/len(ytrue)
                
        tprs.append(row_tps)
        fprs.append(row_fps)
    
    return tprs, fprs
        
        

In [30]:
tpr, fpr = evaluate_autotag(true_tags, tag_predictions)

accuracy = sum(tpr)/sum(fpr)
accuracy

0.025844391999071459

### There is a lot of work to be done to improve my autotagging scheme. Here are some components to approve:

- Cosine Similarity threshold for identify tag candidates -- Bayesian Methods

- Filtering for labeled tags in Doc2Vec embedding.

- Classify document thread, then search for tags in Doc2Vec embedding tailored for a specific domain